# Notes on modifying xarray
I was looking at ways to extend xarray to add some functionality specific to ApRES. 

I tried inheriting the class and this seemed very complicated. Then I saw [this](https://github.com/pydata/xarray/issues/2788) and the suggested way is to use accessors. This looks interested, but it turns out that I dont think I need to do this right at the moment because I can stack ApRES data as profiles (if we are careful).

In [23]:
import xarray as xr
ds = xr.Dataset()


In [3]:
type(ds)

xarray.core.dataset.Dataset

In [111]:
import xarray
class ApRES_xarray(xarray.core.dataset.Dataset):
    def __init__(
        self,
        data_vars=None,
        coords=None,
        attrs=None, extra_data=1):
        super().__init__(
                data_vars,
                coords,
                attrs)
        
        self.extra_data = extra_data
            
        #   pass
        #print(self)
        #self['var1'] = data
    def disp_mean(self):
        print(self.mean())
        
    

In [114]:
temp =ApRES_xarray([1])

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [70]:
import numpy as np
import pandas as pd
np.random.seed(0)
temperature = 15 + 8 * np.random.randn(2, 2, 3)
precipitation = 10 * np.random.rand(2, 2, 3)
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]
time = pd.date_range("2014-09-06", periods=3)
reference_time = pd.Timestamp("2014-09-05")

In [79]:
ds = ApRES_xarray(
    data_vars=dict(
        temperature=(["x", "y", "time"], temperature),
        precipitation=(["x", "y", "time"], precipitation),
    ),
    coords=dict(
        lon=(["x", "y"], lon),
        lat=(["x", "y"], lat),
        time=time,
        reference_time=reference_time,
    ),
    attrs=dict(description="Weather related data."),
)

TypeError: ApRES_xarray.__init__() got an unexpected keyword argument 'data_vars'

In [76]:
ds.disp_mean()

<xarray.ApRES_xarray>
Dimensions:         ()
Coordinates:
    reference_time  datetime64[ns] 2014-09-05
Data variables:
    temperature     float64 20.99
    precipitation   float64 5.977


In [83]:
xr.Dataset.__file__


AttributeError: type object 'Dataset' has no attribute '__file__'

In [119]:
import xarray as xr


@xr.register_dataset_accessor("apres")
class apresAccessor:
    def __init__(self, xarray_obj):
        self._obj = xarray_obj
        self._center = None

    @property
    def center(self):
        """Return the geographic center point of this dataset."""
        if self._center is None:
            # we can use a cache on our accessor objects, because accessors
            # themselves are cached on instances that access them.
            lon = self._obj.latitude
            lat = self._obj.longitude
            self._center = (float(lon.mean()), float(lat.mean()))
        return self._center

    def plot(self):
        """Plot data on a map."""
        return "plotting!"


In [120]:
ds = xr.Dataset({"longitude": np.linspace(0, 10), "latitude": np.linspace(0, 20)})


In [127]:

ds.apres.center


(10.0, 5.0)

In [129]:
ds.lat

AttributeError: 'Dataset' object has no attribute 'lat'

In [118]:

ds.geo.plot()


'plotting!'